In [ ]:
using Revise, Genie, DelimitedFiles, DCAUtils, JLD2, PyPlot, Statistics, LinearAlgebra
import KitMSA: fasta2matrix

nat_msa  = read_fasta_alignment("../data_Genie/DBD_independent_small_d_alignment.jld2", 0.9);
dbd_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m",0.9);
w_lore_real = readdlm("../data_Genie/independent_small_d_alignment_weights.dat")[:];
q = 21; L = size(nat_msa,1);

lore_msa =Int8.(fasta2matrix("../data_Genie/sampling_samples.fasta", max_gap_fraction = 0.9)');

Genie.conn_corr(lore_msa, nat_msa, q, L, nat_weight = 0.2, sim_weight = 0.)
Genie.conn_corr_with_pc(lore_msa, nat_msa, q, L, pc = 1e-6, nat_weight = 0.2, sim_weight = 0.)
Genie.conn_corr_with_weight(lore_msa, nat_msa, q, L, w_lore_real)


J_tmp, h_tmp, alf = Genie.read_graph_new("../data_Genie/independent_small_d_alignment_params.dat"
    );J_tmp_rs = reshape(J_tmp, q,L,q,L); h = Float64.(circshift(h_tmp, (-1, 0))); J = Float64.(circshift(J_tmp_rs, (-1, 0, -1, 0)));


start_msa = hcat([nat_msa[:,1] for i in 1:10000]...); 

#start_msa = nat_msa[:,rand(1:size(nat_msa,2), 1000)]; 

@time res = run_evolution(start_msa, 
    h, 
    J,
    p = 0.95, 
    temp = 1., 
    N_steps = 4*10^5,  
    N_points = 20, 
    verbose = true);

Genie.conn_corr_with_weight(res.step_msa[end], nat_msa, q, L, w_lore_real)
Genie.conn_corr(res.step_msa[end], nat_msa, q, L, nat_weight = 0.2, sim_weight = 0.)
@time Genie.check_equilibration_with_weight("../figures/small_dbd_80_id/", nat_msa, res.step_msa, res.steps, w_lore_real, nat_weight = 0.2, sim_weight = 0.)

@time res_metrop = run_potts(start_msa, 
    h, 
    J,
    temp = 1., 
    N_steps = 6*10^4,  
    N_points = 20, 
    verbose = true);


Genie.conn_corr_with_weight(res_metrop.step_msa[end], nat_msa, q, L, w_lore_real)
Genie.conn_corr(res_metrop.step_msa[end], nat_msa, q, L, nat_weight = 0.2, sim_weight = 0.)



@time Genie.check_equilibration_with_weight("../figures/small_dbd_80_id_metrop/", nat_msa, res_metrop.step_msa, res_metrop.steps,  w_lore_real, nat_weight = 0.2, sim_weight = 0.)




Precompiling Genie
  ✓ Genie
  1 dependency successfully precompiled in 15 seconds. 158 already precompiled.


LoadError: SystemError: ../data_Genie/DBD_independent_small_d_alignment.jld2: No such file or directory

In [ ]:
#start_msa = nat_msa[:,rand(1:size(dbd_msa,2),5000)];
start_msa = hcat([dbd_msa[:,16190] for _ in 1:5000]...);
@time res0_metrop = run_potts(start_msa, 
    h_dbd, 
    J_dbd,
    temp = 1., 
    N_steps = 50);


#start_msa = hcat([rand(1:21,76) for _ in 1:5000]...);
start_msa = res0_metrop.msa
@time res_metrop = run_potts(start_msa, 
    h, 
    J,
    temp = 1., 
    N_steps = 3*10^5,  
    N_points = 200, 
    verbose = true);


Genie.conn_corr_with_weight(res_metrop.step_msa[end], nat_msa, q, L, w_lore_real)
@time Genie.check_equilibration_with_weight("../figures/small_dbd_80_id_metrop/", nat_msa, res_metrop.step_msa, res_metrop.steps,  w_lore_real, nat_weight = 0.2, sim_weight = 0.)



step_msa = res_metrop.step_msa;
w = w_lore_real;
train_msa = nat_msa;

L, M = size(step_msa[1]); T = length(step_msa); q = 21; 
f1,f2 = compute_weighted_frequencies(train_msa, w, 22);
f1 = reshape(f1, (21,76));
f1_0,f2_0 = compute_weighted_frequencies(step_msa[1], 22, 0.);
f1_0 = reshape(f1_0, (21,76));
p1 = zeros(M, 21, L);
@time for idx in 1:M
    for a in 1:21
        for i in 1:L
            p1[idx,:,i] = Genie.single_site_prob_cond(i,step_msa[1][:,idx],h,J, 76)
        end
    end
end

p1_0 = mean(p1, dims = 1)[1,:,:];
scores = (Genie.pseudocount1(f1,1e-6,q) .- Genie.pseudocount1(p1_0,1e-6,q)) ./ (Genie.pseudocount1(f1,
        1e-5,q) .- Genie.pseudocount1(f1_0,1e-6,q));

freqs = zeros(T, 21, L);
for i in 1:T
     @time freqs[i, :, :] = reshape(compute_weighted_frequencies(step_msa[i], 22, 0.)[1], (21,L))
end

times = zeros(21,L);
prefac = zeros(21,L);
x = res_metrop.steps ./L;
@time for a in 1:21
    println(a)
    for i in 1:76
        model(t, p) = p[1] *(1 .- exp.(-t/p[2])) 
        p0 = [0.5, 0.5]
        fiti = curve_fit(model, x, freqs[:,a,i], p0);
        prefac[a,i] = fiti.param[1]
        times[a,i] = fiti.param[2]
    end
end
cor(scores[:], times[:])

a = 2; site = 53; close("all"); x = res_metrop.steps; plt.plot(x, freqs[:,a,site], 
    label = "sim freq"); plt.plot(x, 
    prefac[a,site] .* (1 .- exp.(.-x ./times[a,site])), label = "fit"); plt.xscale("log");plt.legend(
    );savefig("../prova_fit.png")


num = f1 .- p1_0; idx = (f1_0 .< 1.) .& (f1 .> 0.) .& (
           num .> -1.); println(sum(idx)); close("all"); plt.scatter(times[idx],num[idx]); plt.xlabel("tau"); plt.ylabel("f1 - p1_0");plt.savefig(
    "../filtered_corr.png");(cor(scores[idx], times[idx]), 
    cor(num[idx], times[idx]))



In [ ]:
cde = get_entropy(p1_0); cie = get_entropy(f1); epist = hcat([cie.-cde for _ in 1:21]...)';
close("all"); plt.scatter(cde,cie); plt.xlabel("<CDE>"); plt.ylabel("CIE"); plt.savefig("../cde_cie.png")

num = f1 .- p1_0; idx = (f1_0 .< 1.) .& (f1 .> 0.) .& (
           num .> -1.); println(sum(idx)); close("all"); plt.scatter(times[idx],num[idx]); plt.xlabel("tau"); plt.ylabel("f1 - p1_0");plt.savefig(
    "../filtered_corr.png");(cor(scores[idx], times[idx]), 
    cor(num[idx], times[idx]))




In [ ]:
pp1 = zeros(M, 21, L);
@time for idx in 1:M
    for a in 1:21
        for i in 1:L
            pp1[idx,:,i] = Genie.single_site_prob_cond(i,train_msa[:,idx],h,J, 76)
        end
    end
end

p1_train = mean(pp1, dims = 1)[1,:,:];